In [1]:
import numpy as np
import numpy.linalg as linalg
import scipy.optimize as op
import scipy.interpolate
from scipy.integrate import quad
import pandas as pd
import time
from cobaya import run
import getdist.plots as gdplt
from getdist import MCSamples
import matplotlib.pyplot as plt
import os

c = 299792.458#km/s

## CC data

In [2]:
zHz, Hzi, errHz = np.genfromtxt('data/32CCdata.dat', comments='#', usecols=(0,1,2), unpack=True, delimiter=',')
zmod, imf, slib, sps, spsooo = np.genfromtxt('data/data_MM20.dat', comments='#', usecols=(0,1,2,3,4), unpack=True)

cov_mat_diag = np.zeros((len(zHz), len(zHz)), dtype='float64') 

for i in range(len(zHz)):
    cov_mat_diag[i,i] = errHz[i]**2#Erros estatísticos

#Erros sistemáticos
imf_intp = np.interp(zHz, zmod, imf)/100
spsooo_intp = np.interp(zHz, zmod, spsooo)/100

cov_mat_imf = np.zeros((len(zHz), len(zHz)), dtype='float64')
cov_mat_spsooo = np.zeros((len(zHz), len(zHz)), dtype='float64')

for i in range(len(zHz)):
    for j in range(len(zHz)):
        cov_mat_imf[i,j] = Hzi[i] * imf_intp[i] * Hzi[j] * imf_intp[j]
        cov_mat_spsooo[i,j] = Hzi[i] * spsooo_intp[i] * Hzi[j] * spsooo_intp[j]
        
#Sistemático+estatístico:
cov_mat = cov_mat_spsooo+cov_mat_imf+cov_mat_diag
inv_cov_mat = np.linalg.inv(cov_mat)

## Pantheon+SH0ES data

In [3]:
def build_data(datafile,SH0ES):
    """
    Run once at the start to load in the data vectors.
    
    Returns x, y where x is the independent variable (redshift in this case)
    and y is the Gaussian-distribured measured variable (magnitude in this case).
    """
    
    global origlen, wwPS, wwPPlus, cepheid_distance, is_calibrator, zHELPS, zHELPPlus

    print("Loading data from {}".format(datafile))
    data = pd.read_csv(datafile, sep=r'\s+')
    origlen = len(data)
    
    if SH0ES:
        ww = (data['zHD']>0.01) | (np.array(data['IS_CALIBRATOR'],dtype=bool))
        wwPS = ww
        zHELPS = data['zHEL'][ww]
    else:
        ww = (data['zHD']>0.01)
        wwPPlus = ww
        zHELPPlus = data['zHEL'][ww]

    zCMB = data['zHD'][ww] #use the vpec corrected redshift for zCMB 
    
    m_obs = data['m_b_corr'][ww]
    if SH0ES:
        is_calibrator = data['IS_CALIBRATOR'][ww]
        cepheid_distance = data['CEPH_DIST'][ww]

    return zCMB, m_obs

def build_covariance(covdatafile, SH0ES):
    """
    Run once at the start to build the covariance matrix for the data
    The file format for the covariance has the first line as an integer
    indicating the number of covariance elements, and the the subsequent
    lines being the elements.
    This function reads in the file and the nasty for loops trim down the covariance
    to match the only rows of data that are used for cosmology    
    """
    print("Loading covariance from {}".format(covdatafile))


    f = open(covdatafile)
    line = f.readline()
    if SH0ES:
        n = int(len(zCMBPS))
        ww = wwPS
    else:
        n = int(len(zCMBPPlus))
        ww = wwPPlus
    C = np.zeros((n,n))
    ii = -1
    jj = -1
    mine = 999
    maxe = -999
    for i in range(origlen):
        jj = -1
        if ww[i]:
            ii += 1
        for j in range(origlen):
            if ww[j]:
                jj += 1
            val = float(f.readline())
            if ww[i]:
                if ww[j]:
                    C[ii,jj] = val
    f.close()
    
    print('Done')

        # Return the covariance; the parent class knows to invert this
        # later to get the precision matrix that we need for the likelihood.
    return C

### Pantheon+SH0ES

In [4]:
zCMBPS, m_obsPS = build_data('data/Pantheon+SH0ES.dat',SH0ES=True)

covPS = build_covariance('data/Pantheon+SH0ES_STAT+SYS.cov', SH0ES=True)
invcovPS = linalg.inv(covPS)

Loading data from data/Pantheon+SH0ES.dat
Loading covariance from data/Pantheon+SH0ES_STAT+SYS.cov
Done


### Pantheon+

In [5]:
zCMBPPlus, m_obsPPlus = build_data('data/Pantheon+SH0ES.dat',SH0ES=False)

covPPlus = build_covariance('data/Pantheon+SH0ES_STAT+SYS.cov', SH0ES=False)
invcovPPlus = linalg.inv(covPPlus)

Loading data from data/Pantheon+SH0ES.dat
Loading covariance from data/Pantheon+SH0ES_STAT+SYS.cov
Done


## Model functions

In [6]:
#Theory points for Pantheon+ and Pantheon+&SH0ES
def extract_theory_points(theory_x, theory_y, pars, SH0ES):
    """
    Run once per parameter set to extract the mean vector that our
    data points are compared to.  For the Hubble flow set, we compare to the 
    cosmological model. For the calibrators we compare to the Cepheid distances.
    """
    M = pars[0]

    # Pull out theory mu and z from the block.
    #theory_x = block[self.x_section, self.x_name]
    #theory_y = block[self.y_section, self.y_name]

    # Interpolation function of theory so we can evaluate at redshifts of the data
    f = scipy.interpolate.interp1d(theory_x, theory_y)#, kind=self.kind)
    
    if SH0ES:
        theory_ynew = zCMBPS*np.nan
        #Here we use the Cepheid host distances as the "theory". 
        #This is the 2nd rung of the distance ladder and is what calibrates M
        theory_ynew[np.array(is_calibrator,dtype='bool')] = cepheid_distance[np.array(is_calibrator,dtype='bool')]
        
        # Actually do the interpolation at the data redshifts
        zcmb = zCMBPS[~np.array(is_calibrator,dtype='bool')]
        zhel = zHELPS[~np.array(is_calibrator,dtype='bool')]
    else:
        theory_ynew = zCMBPPlus*np.nan
        # Actually do the interpolation at the data redshifts
        zcmb = zCMBPPlus
        zhel = zHELPPlus
    
    fz = f(zcmb)
    if np.any(fz<=0):
        NegDist = True
        return 1
    if SH0ES:
        theory_ynew[~np.array(is_calibrator,dtype='bool')] = 5.0*np.log10((1.0+zcmb)*(1.0+zhel)*\
                                                                          np.atleast_1d(fz))+25.
    else:
        theory_ynew = 5.0*np.log10((1.0+zcmb)*(1.0+zhel)*np.atleast_1d(fz))+25.
                
    # Add the absolute supernova magnitude and return
    #M = block[names.supernova_params, "M"]
    return theory_ynew + M

def extract_theory_pointsHz(theory_x, theory_y, pars):
    theory_ynew = zHz*np.nan  

    f = scipy.interpolate.interp1d(theory_x, theory_y)#, kind=self.kind)
    theory_ynew = np.atleast_1d(f(zHz))
    return theory_ynew

#Points for theory interpolation
N = 3000
zt = np.linspace(0,2.5,N)

### P21

In [7]:
def EzP21(z,theta):
    M, H0, q0, j0 = theta
    a0 = H0
    a1 = (H0*(6 + 10*j0 + 12*q0 + 7*j0*q0 - 3*q0**2 - 6*q0**3))/(3*(j0 - q0**2))
    a2 = (H0*(12 + 14*j0 + 3*j0**2 + 36*q0 + 22*j0*q0 + 24*q0**2 + 2*j0*q0**2 - 6*q0**3 - 3*q0**4))/(6*(j0 - q0**2))
    b1 = (6 + 7*j0 + 12*q0 + 4*j0*q0 - 3*q0**3)/(3*(j0 - q0**2))
        
    H = (a0+a1*z+a2*z**2)/(1+b1*z)

    return H/H0

#Distancia comovel adimensional
def DcModelP21(z,theta):#Integrando e substituindo
    M, H0, q0, j0 = theta
    a0 = H0
    a1 = (H0*(6 + 10*j0 + 12*q0 + 7*j0*q0 - 3*q0**2 - 6*q0**3))/(3*(j0 - q0**2))
    a2 = (H0*(12 + 14*j0 + 3*j0**2 + 36*q0 + 22*j0*q0 + 24*q0**2 + 2*j0*q0**2 - 6*q0**3 - 3*q0**4))/(6*(j0 - q0**2))
    b1 = (6 + 7*j0 + 12*q0 + 4*j0*q0 - 3*q0**3)/(3*(j0 - q0**2))

    if (a1**2-4*a0*a2) > 0:
        r1 = (-a1 + np.sqrt(a1**2-4*a2*a0))/(2*a2)
        r2 = (-a1 - np.sqrt(a1**2-4*a2*a0))/(2*a2)

        dc = (((1+b1*r2)/(r2-r1))*np.log(np.abs((z-r2)/(-r2)))-((1+b1*r1)/(r2-r1))*np.log(np.abs((z-r1)/(-r1))))/a2
        
    elif (a1**2-4*a0*a2) == 0:
        dc = ((1 + (a1*b1)/(2*a2))*2*a2*z)/(a1*(a1/(2*a2) + z)) + (b1*np.log(2*np.abs((a2*(a1/(2*a2) + z))/a1)))/a2
        
    elif (a1**2-4*a0*a2) < 0:
        dc = (b1/(2*a2))*np.log(np.abs((a0+a1*z+a2*z**2)/a0)) + (((2*a2-b1*a1))/(a2*np.sqrt(4*a2*a0-a1**2)))*(np.arctan((2*a2*z+a1)/np.sqrt(4*a2*a0-a1**2)) - np.arctan(a1/np.sqrt(4*a2*a0-a1**2)))

    return H0*dc

#Distancia comovel transversal in pc
def dmModelP21(z, theta):
    M, H0, q0, j0 = theta
    
    dH = c/H0
    wk = 0
    Dcm = DcModelP21(z, theta)
    if wk < 0:
        return dH*np.sin(np.sqrt(-wk)*Dcm)/np.sqrt(-wk)
    elif wk > 0:
        return dH*np.sinh(np.sqrt(wk)*Dcm)/np.sqrt(wk)
    return dH*Dcm

#Distancia comovel transversal in pc
def dmModelP21(z, theta):
    M, H0, q0, j0 = theta
    
    dH = c/H0
    wk = 0
    Dcm = DcModelP21(z, theta)
    if wk < 0:
        return dH*np.sin(np.sqrt(-wk)*Dcm)/np.sqrt(-wk)
    elif wk > 0:
        return dH*np.sinh(np.sqrt(wk)*Dcm)/np.sqrt(wk)
    return dH*Dcm

#distancia de luminosidade in pc
def dlModelP21(z,theta):
    return (1+z)*dmModelP21(z,theta)

def daModelP21(z,theta):
    return dmModelP21(z,theta)/(1+z)

def muModelP21(z,pars):
    return 5.*np.log10(dlModelP21(z,pars))+25

### P22

In [8]:
def EzP22(z, theta):
    M, H0, q0, j0, s0 = theta
    a0 = H0
    a1 = (H0*(-24 + 4*j0 + 8*j0**2 - 84*q0 + 54*j0*q0 + 10*j0**2*q0 - 114*q0**2 + 67*j0*q0**2 - 102*q0**3 + 15*j0*q0**3 - 63*q0**4 - 15*q0**5
              + 17*s0 + 2*j0*s0 + 28*q0*s0 + 9*q0**2*s0))/(2.*(6*j0 + 3*j0**2 + 14*j0*q0 - 6*q0**2 + 2*j0*q0**2 - 12*q0**3 - 3*q0**4 + 2*s0 + 2*q0*s0))
    a2 = (H0*(-144 + 24*j0 + 108*j0**2 + 30*j0**3 - 648*q0 + 216*j0*q0 + 150*j0**2*q0 - 1188*q0**2 + 252*j0*q0**2 - 5*j0**2*q0**2 - 1188*q0**3
              + 90*j0*q0**3 - 648*q0**4 + 60*j0*q0**4 - 180*q0**5 - 45*q0**6 + 78*s0 + 9*j0*s0 + 198*q0*s0 + 35*j0*q0*s0 + 153*q0**2*s0 +
              15*q0**3*s0 + 4*s0**2))/(12.*(6*j0 + 3*j0**2 + 14*j0*q0 - 6*q0**2 + 2*j0*q0**2 - 12*q0**3 - 3*q0**4 + 2*s0 + 2*q0*s0))
    b1 = (24 + 8*j0 - 2*j0**2 + 84*q0 - 14*j0*q0 - 4*j0**2*q0 + 102*q0**2 - 35*j0*q0**2 + 66*q0**3 - 11*j0*q0**3 + 33*q0**4 + 9*q0**5 -
              13*s0 - 2*j0*s0 - 20*q0*s0 - 5*q0**2*s0)/(-12*j0 - 6*j0**2 - 28*j0*q0 + 12*q0**2 - 4*j0*q0**2 + 24*q0**3 + 6*q0**4 - 4*s0 - 4*q0*s0)
    b2 = (-72*j0 - 60*j0**2 - 12*j0**3 - 180*j0*q0 - 30*j0**2*q0 + 72*q0**2 - 30*j0*q0**2 + 23*j0**2*q0**2 + 180*q0**3 + 30*j0*q0**3 +
          90*q0**4 - 24*j0*q0**4 + 9*q0**6 + 15*j0*s0 - 11*j0*q0*s0 - 15*q0**2*s0 + 3*q0**3*s0 - 4*s0**2)/(-72*j0 - 36*j0**2 - 168*j0*q0 
                                                                                                           + 72*q0**2 - 24*j0*q0**2 + 144*q0**3 + 36*q0**4 - 24*s0 - 24*q0*s0)
        
    H = (a0+a1*z+a2*z**2)/(1+b1*z+b2*z**2)

    return H/H0

def DcModelP22(z, theta):#Integrando e substituindo
    M, H0, q0, j0, s0 = theta
    a0 = H0
    a1 = (H0*(-24 + 4*j0 + 8*j0**2 - 84*q0 + 54*j0*q0 + 10*j0**2*q0 - 114*q0**2 + 67*j0*q0**2 - 102*q0**3 + 15*j0*q0**3 - 63*q0**4 - 15*q0**5
              + 17*s0 + 2*j0*s0 + 28*q0*s0 + 9*q0**2*s0))/(2.*(6*j0 + 3*j0**2 + 14*j0*q0 - 6*q0**2 + 2*j0*q0**2 - 12*q0**3 - 3*q0**4 + 2*s0 + 2*q0*s0))
    a2 = (H0*(-144 + 24*j0 + 108*j0**2 + 30*j0**3 - 648*q0 + 216*j0*q0 + 150*j0**2*q0 - 1188*q0**2 + 252*j0*q0**2 - 5*j0**2*q0**2 - 1188*q0**3
              + 90*j0*q0**3 - 648*q0**4 + 60*j0*q0**4 - 180*q0**5 - 45*q0**6 + 78*s0 + 9*j0*s0 + 198*q0*s0 + 35*j0*q0*s0 + 153*q0**2*s0 +
              15*q0**3*s0 + 4*s0**2))/(12.*(6*j0 + 3*j0**2 + 14*j0*q0 - 6*q0**2 + 2*j0*q0**2 - 12*q0**3 - 3*q0**4 + 2*s0 + 2*q0*s0))
    b1 = (24 + 8*j0 - 2*j0**2 + 84*q0 - 14*j0*q0 - 4*j0**2*q0 + 102*q0**2 - 35*j0*q0**2 + 66*q0**3 - 11*j0*q0**3 + 33*q0**4 + 9*q0**5 -
              13*s0 - 2*j0*s0 - 20*q0*s0 - 5*q0**2*s0)/(-12*j0 - 6*j0**2 - 28*j0*q0 + 12*q0**2 - 4*j0*q0**2 + 24*q0**3 + 6*q0**4 - 4*s0 - 4*q0*s0)
    #Coeficiente b2 correto
    b2 = (-72*j0 - 60*j0**2 - 12*j0**3 - 180*j0*q0 - 30*j0**2*q0 + 72*q0**2 - 30*j0*q0**2 + 23*j0**2*q0**2 + 180*q0**3 + 30*j0*q0**3 +
          90*q0**4 - 24*j0*q0**4 + 9*q0**6 + 15*j0*s0 - 11*j0*q0*s0 - 15*q0**2*s0 + 3*q0**3*s0 - 4*s0**2)/(-72*j0 - 36*j0**2 - 168*j0*q0
              + 72*q0**2 - 24*j0*q0**2 + 144*q0**3 + 36*q0**4 - 24*s0 - 24*q0*s0)
    #Coeficiente b2 errado?
    #b2 = (-72*j0 - 60*j0**2 - 12*j0**3 - 180*j0*q0 - 30*j0**2*q0 + 72*q0**2 - 30*j0*q0**2 + 23*j0**2*q0**2 + 180*q0**3 + 30*j0*q0**3 +
    #      90*q0**4 - 24*j0*q0**4 + 9*q0**6 + 15*j0*s0 - 11*j0*q0*s0 - 15*q0**2*s0 + 3*q0**3*s0 - 4*s0**2)/(-72*j0 - 36*j0**2 - 168*j0*q0)

    if (a1**2-4*a0*a2) > 0:
        r1 = (-a1 + np.sqrt(a1**2-4*a2*a0))/(2*a2)
        r2 = (-a1 - np.sqrt(a1**2-4*a2*a0))/(2*a2)
        dc = (((1 + b1*r2 + r2**2)/(r2 - r1))*np.log(np.abs((z - r2)/(-r2))) - ((1 + b1*r1 + r1**2)/(r2 - r1))*np.log(np.abs((z-r1)/(-r1))) + z)/a2
        
    elif (a1**2-4*a0*a2) == 0:
        dc = ((b1 - (b2*a1)/(2*a2))*np.log(np.abs((2*a2*z + a1)/a1)) - (1 - (b1*a1)/(2*a2) + (b2*a1)/(4*a2**2))*(1/(a1*(2*a2*z +a1)) - b2)*z)/a2
        
    elif (a1**2-4*a0*a2) < 0:
        dc = ((b2*z)/a2 + (b1/(2*a2) - (b2*a1)/(2*a2**2))*np.log(np.abs((a0+a1*z+a2*z**2)/a0)) + (((2*a2-b1*a1 - b2*(2*a2*a0 - a1**2)/a2))/(a2*np.sqrt(4*a2*a0-a1**2)))*(np.arctan((2*a2*z+a1)/np.sqrt(4*a2*a0-a1**2))
                    - np.arctan(a1/np.sqrt(4*a2*a0-a1**2))))/a2

    return H0*dc

#Distancia comovel transversal in pc
def dmModelP22(z, theta):
    M, H0, q0, j0, s0 = theta
    
    dH = c/H0
    wk = 0
    Dcm = DcModelP22(z, theta)
    if wk < 0:
        return dH*np.sin(np.sqrt(-wk)*Dcm)/np.sqrt(-wk)
    elif wk > 0:
        return dH*np.sinh(np.sqrt(wk)*Dcm)/np.sqrt(wk)
    return dH*Dcm

#distancia de luminosidade in pc
def dlModelP22(z,theta):
    return (1+z)*dmModelP22(z,theta)

def daModelP22(z,theta):
    return dmModelP22(z,theta)/(1+z)

def muModelP22(z,pars):
    return 5.*np.log10(dlModelP22(z,pars))+25

### P31

In [9]:
def EzP31(z, theta):
    M, H0, q0, j0, s0 = theta
    a0 = H0
    a1 = -0.25*(H0*(24 - 4*j0 + 4*j0**2 + 60*q0 - 50*j0*q0 + 54*q0**2 - 41*j0*q0**2 + 48*q0**3 + 27*q0**4 - 17*s0 - 11*q0*s0))/(3*j0 + 4*j0*q0 - 3*q0**2 - 3*q0**3 + s0)
    a2 = (H0*(-24 - 8*j0 + 2*j0**2 - 84*q0 + 14*j0*q0 + 4*j0**2*q0 - 102*q0**2 + 35*j0*q0**2 - 66*q0**3 + 
              11*j0*q0**3 - 33*q0**4 - 9*q0**5 + 13*s0 + 2*j0*s0 + 20*q0*s0 + 5*q0**2*s0))/(4.*(3*j0 + 4*j0*q0 - 3*q0**2 - 3*q0**3 + s0))
    a3 = -0.0417*(H0*(72*j0 + 60*j0**2 + 12*j0**3 + 180*j0*q0 + 30*j0**2*q0 - 72*q0**2 + 30*j0*q0**2 - 
                      23*j0**2*q0**2 - 180*q0**3 - 30*j0*q0**3 - 90*q0**4 + 24*j0*q0**4 - 9*q0**6 - 15*j0*s0 + 11*j0*q0*s0 + 
                      15*q0**2*s0 - 3*q0**3*s0 + 4*s0**2))/(3*j0 + 4*j0*q0 - 3*q0**2 - 3*q0**3 + s0) #0.041666666666666664
    b1 = (-24 - 8*j0 - 4*j0**2 - 60*q0 + 22*j0*q0 - 42*q0**2 + 25*j0*q0**2 - 24*q0**3 - 15*q0**4 + 13*s0 + 7*q0*s0)/(4.*(3*j0 + 4*j0*q0 - 3*q0**2 - 3*q0**3 + s0))

    H = (a0 + a1*z + a2*z**2 + a3*z**3)/(1 + b1*z)

    return H/H0

#Distancia comovel adimensional
def DcModelP31(z,theta):#Integrando e substituindo
    M, H0, q0, j0, s0 = theta
    dc = np.zeros_like(z)
    
    a0 = H0
    a1 = -0.25*(H0*(24 - 4*j0 + 4*j0**2 + 60*q0 - 50*j0*q0 + 54*q0**2 - 41*j0*q0**2 + 48*q0**3 + 27*q0**4 - 17*s0 - 11*q0*s0))/(3*j0 + 4*j0*q0 - 3*q0**2 - 3*q0**3 + s0)
    
    a2 = (H0*(-24 - 8*j0 + 2*j0**2 - 84*q0 + 14*j0*q0 + 4*j0**2*q0 - 102*q0**2 + 35*j0*q0**2 - 66*q0**3 + 11*j0*q0**3 - 
              33*q0**4 - 9*q0**5 + 13*s0 + 2*j0*s0 + 20*q0*s0 + 5*q0**2*s0))/(4.*(3*j0 + 4*j0*q0 - 3*q0**2 - 3*q0**3 + s0))
    
    a3 = -0.0417*(H0*(72*j0 + 60*j0**2 + 12*j0**3 + 180*j0*q0 + 30*j0**2*q0 - 72*q0**2 + 30*j0*q0**2 - 
                      23*j0**2*q0**2 - 180*q0**3 - 30*j0*q0**3 - 90*q0**4 + 24*j0*q0**4 - 9*q0**6 - 15*j0*s0 + 11*j0*q0*s0 + 
                      15*q0**2*s0 - 3*q0**3*s0 + 4*s0**2))/(3*j0 + 4*j0*q0 - 3*q0**2 - 3*q0**3 + s0) #0.041666666666666664
    
    b1 = (-24 - 8*j0 - 4*j0**2 - 60*q0 + 22*j0*q0 - 42*q0**2 + 25*j0*q0**2 - 24*q0**3 - 15*q0**4 + 13*s0 + 7*q0*s0)/(4.*(3*j0 + 4*j0*q0 - 3*q0**2 - 3*q0**3 + s0))

    for i in range(len(z)):
        dc[i], _ = quad(EzP31, 0, z[i], args=(theta))
        if not np.isfinite(dc[i]):
            dc[i] = np.inf

    return (H0)*dc

#Distancia comovel transversal in pc
def dmModelP31(z, theta):
    M, H0, q0, j0, s0= theta
    
    dH = c/H0
    wk = 0
    Dcm = DcModelP31(z, theta)
    if wk < 0:
        return dH*np.sin(np.sqrt(-wk)*Dcm)/np.sqrt(-wk)
    elif wk > 0:
        return dH*np.sinh(np.sqrt(wk)*Dcm)/np.sqrt(wk)
    return dH*Dcm

#distancia de luminosidade in pc
def dlModelP31(z,theta):
    return (1+z)*dmModelP31(z,theta)

def daModelP31(z,theta):
    return dmModelP31(z,theta)/(1+z)

def muModelP31(z,pars):
    return 5.*np.log10(dlModelP31(z,pars))+25

## Statistical functions

### P21

In [10]:
def getpar_PSHz_21(theta):
    M, h0, q0, j0 = theta
    return lnprobPSHz_21(M, h0, q0, j0)

def getpar_PPHz_21(theta):
    M, h0, q0, j0 = theta
    return lnprobPPHz_21(M, h0, q0, j0)

def chi2PS_21(theta):#Pantheon+SH0ES
    zsni = zCMBPS
    Ez2i = EzP21(zsni,theta)
    if np.any(Ez2i <= 0):
        return np.inf

    dAt = daModelP21(zt,theta)
    NegDist = False
    mmod = extract_theory_points(zt, dAt, theta, SH0ES=True)
    if NegDist:
        return np.inf
    dm = mmod-m_obsPS
    
    return np.dot(np.dot(dm.T,invcovPS),dm)
        
def chi2PP_21(theta):#Pantheon+
    zsni = zCMBPPlus
    Ez2i = EzP21(zsni,theta)
    if np.any(Ez2i <= 0):
        return np.inf

    dAt = daModelP21(zt,theta)
    NegDist = False
    mmod = extract_theory_points(zt, dAt, theta,SH0ES=False) #5.*np.log10((1+zsni)*(1+zheli)*dAmodi/10)#(dlmodi/10)
    if NegDist:
        return np.inf
    dmu = mmod-m_obsPPlus
    SA  = np.sum(invcovPPlus)
    Sr  = np.sum(np.dot(dmu.T,invcovPPlus))#np.sum(dmu*invcov)
    Srr = np.dot(np.dot(dmu.T,invcovPPlus),dmu)#np.sum(dmu*dmu*invcov)
    return Srr - Sr**2./SA
        
def chi2Hz_21(theta):#CC Hz
    M, H0, q0, j0 = theta
        
    Ezt = EzP21(zt,theta)
    if np.any(Ezt<0):
        return np.inf
    Hzt = H0*Ezt#Hz(zt,pars)
    Hzm = extract_theory_pointsHz(zt, Hzt, theta)
   
    dh = Hzm-Hzi
    
    return np.dot(np.dot(dh.T,inv_cov_mat),dh)

def lnlikePS_21(theta):
    return -0.5*chi2PS_21(theta)

def lnlikePP_21(theta):
    return -0.5*chi2PP_21(theta)

def lnlikeHz_21(theta):
    return -0.5*chi2Hz_21(theta)

def lnprobPSHz_21(M, h0, q0, j0):
    theta = [M, h0, q0, j0]
    return lnlikePS_21(theta)+lnlikeHz_21(theta)
    
def lnprobPPHz_21(M, h0, q0, j0):
    theta = [M, h0, q0, j0]
    return lnlikePP_21(theta)+lnlikeHz_21(theta)

### P22

In [11]:
def getpar_PSHz_22(theta):
    M, h0, q0, j0, s0 = theta
    return lnprobPSHz_22(M, h0, q0, j0, s0)

def getpar_PPHz_22(theta):
    M, h0, q0, j0, s0 = theta
    return lnprobPPHz_22(M, h0, q0, j0, s0)

def chi2PS_22(theta):#Pantheon+SH0ES
    zsni = zCMBPS
    Ez2i = EzP22(zsni,theta)
    if np.any(Ez2i <= 0):
        return np.inf

    dAt = daModelP22(zt,theta)
    NegDist = False
    mmod = extract_theory_points(zt, dAt, theta, SH0ES=True)
    if NegDist:
        return np.inf
    dm = mmod-m_obsPS
    
    return np.dot(np.dot(dm.T,invcovPS),dm)
        
def chi2PP_22(theta):#Pantheon+
    zsni = zCMBPPlus
    Ez2i = EzP22(zsni,theta)
    if np.any(Ez2i <= 0):
        return np.inf

    dAt = daModelP22(zt,theta)
    NegDist = False
    mmod = extract_theory_points(zt, dAt, theta,SH0ES=False) #5.*np.log10((1+zsni)*(1+zheli)*dAmodi/10)#(dlmodi/10)
    if NegDist:
        return np.inf
    dmu = mmod-m_obsPPlus
    SA  = np.sum(invcovPPlus)
    Sr  = np.sum(np.dot(dmu.T,invcovPPlus))#np.sum(dmu*invcov)
    Srr = np.dot(np.dot(dmu.T,invcovPPlus),dmu)#np.sum(dmu*dmu*invcov)
    return Srr - Sr**2./SA
        
def chi2Hz_22(theta):#CC Hz
    M, H0, q0, j0, s0 = theta
        
    Ezt = EzP22(zt,theta)
    if np.any(Ezt<0):
        return np.inf
    Hzt = H0*Ezt#Hz(zt,pars)
    Hzm = extract_theory_pointsHz(zt, Hzt, theta)
   
    dh = Hzm-Hzi
    
    return np.dot(np.dot(dh.T,inv_cov_mat),dh)

def lnlikePS_22(theta):
    return -0.5*chi2PS_22(theta)

def lnlikePP_22(theta):
    return -0.5*chi2PP_22(theta)

def lnlikeHz_22(theta):
    return -0.5*chi2Hz_22(theta)

def lnprobPSHz_22(M, h0, q0, j0, s0):
    theta = [M, h0, q0, j0, s0]
    return lnlikePS_22(theta)+lnlikeHz_22(theta)
    
def lnprobPPHz_22(M, h0, q0, j0, s0):
    theta = [M, h0, q0, j0, s0]
    return lnlikePP_22(theta)+lnlikeHz_22(theta)

### P31

In [12]:
def getpar_PSHz_31(theta):
    M, h0, q0, j0, s0 = theta
    return lnprobPSHz_31(M, h0, q0, j0, s0)

def getpar_PPHz_31(theta):
    M, h0, q0, j0, s0 = theta
    return lnprobPPHz_31(M, h0, q0, j0, s0)

def chi2PS_31(theta):#Pantheon+SH0ES
    zsni = zCMBPS
    Ez2i = EzP31(zsni,theta)
    if np.any(Ez2i <= 0):
        return np.inf

    dAt = daModelP31(zt,theta)
    NegDist = False
    mmod = extract_theory_points(zt, dAt, theta, SH0ES=True)
    if NegDist:
        return np.inf
    dm = mmod-m_obsPS
    
    return np.dot(np.dot(dm.T,invcovPS),dm)
        
def chi2PP_31(theta):#Pantheon+
    zsni = zCMBPPlus
    Ez2i = EzP31(zsni,theta)
    if np.any(Ez2i <= 0):
        return np.inf

    dAt = daModelP31(zt,theta)
    NegDist = False
    mmod = extract_theory_points(zt, dAt, theta,SH0ES=False) #5.*np.log10((1+zsni)*(1+zheli)*dAmodi/10)#(dlmodi/10)
    if NegDist:
        return np.inf
    dmu = mmod-m_obsPPlus
    SA  = np.sum(invcovPPlus)
    Sr  = np.sum(np.dot(dmu.T,invcovPPlus))#np.sum(dmu*invcov)
    Srr = np.dot(np.dot(dmu.T,invcovPPlus),dmu)#np.sum(dmu*dmu*invcov)
    return Srr - Sr**2./SA
        
def chi2Hz_31(theta):#CC Hz
    M, H0, q0, j0, s0 = theta
        
    Ezt = EzP31(zt,theta)
    if np.any(Ezt<0):
        return np.inf
    Hzt = H0*Ezt#Hz(zt,pars)
    Hzm = extract_theory_pointsHz(zt, Hzt, theta)
   
    dh = Hzm-Hzi
    
    return np.dot(np.dot(dh.T,inv_cov_mat),dh)

def lnlikePS_31(theta):
    return -0.5*chi2PS_31(theta)

def lnlikePP_31(theta):
    return -0.5*chi2PP_31(theta)

def lnlikeHz_31(theta):
    return -0.5*chi2Hz_31(theta)

def lnprobPSHz_31(M, h0, q0, j0, s0):
    theta = [M, h0, q0, j0, s0]
    return lnlikePS_31(theta)+lnlikeHz_31(theta)
    
def lnprobPPHz_31(M, h0, q0, j0, s0):
    theta = [M, h0, q0, j0, s0]
    return lnlikePP_31(theta)+lnlikeHz_31(theta)

## Analysis functions

In [13]:
def find_bestfit(lnlike, parnames, par_ml):#,data):
    t1 = time.time()
    ndim = len(par_ml)
    chi2 = lambda *args: -2 * lnlike(*args)
    result = op.minimize(chi2, par_ml)#, args=data)
    if not result['success']:
        result = op.minimize(chi2, par_ml, method='Nelder-Mead',options={'maxiter': 10000})#, args=data
    par_ml = result["x"]
    print('Maximum likelihood result:')
    for i in range(ndim):
        print(parnames[i],' = ',par_ml[i])
    print('chi2min =',result['fun'])
    #ndof = len(data[0])-ndim
    #print('chi2red =',result['fun']/ndof)
    #print(result)
    t2 = time.time()
    print("tempo total: {0:5.3f} seg".format(t2-t1))
    return result

def run_cobaya(info, info_post):
    print("----- INICIANDO SAMPLER -----")
    t1 = time.time()

    # Roda o sampler
    updated_info, sampler = run(info)

    # Retira o início 
    print("----- RETIRANDO O INICIO -----")
    updated_info_post, sampler_post = run(info_post)

    print(f"Tempo de execução: {time.time()-t1} s")

    return sampler, sampler_post

def MCResult_cobaya(sampler):
    gdsamples = sampler.products()["sample"][0].to_getdist()
    stats = gdsamples.getMargeStats()

    for par in stats.names[:gdsamples.samples.shape[1]-2]:
        par_stats = stats.parWithName(par.name)
        mean = par_stats.mean

        # Erros de 1-sigma (68% CL)
        lower_1s = par_stats.limits[0].lower
        upper_1s = par_stats.limits[0].upper
        err_plus_1s = upper_1s - mean
        err_minus_1s = mean - lower_1s

        # Erros de 2-sigma (95% CL)
        lower_2s = par_stats.limits[1].lower
        upper_2s = par_stats.limits[1].upper
        err_plus_2s = upper_2s - mean
        err_minus_2s = mean - lower_2s
            
        label = f"{par.name}"
        
            
        print(f"{label} = {mean:.4f} +{err_plus_1s:.4f} +{err_plus_2s:.4f} -{err_plus_1s:.4f} -{err_plus_2s:.4f}") 
    
def plot_getdist(samples, params, legends, width=8, fill=False):
    gsamples = []
    for i in samples:
        gsamples.append(i.products()["sample"][0].to_getdist())

    g = gdplt.getSubplotPlotter(width_inch=width)
    g.triangle_plot(gsamples, params, filled=fill, legend_labels=legends)
    plt.show()

def lnlikePPHz_21(M, h0, q0, j0):
    theta = [M, h0, q0, j0]
    return lnlikePP_21(theta)+lnlikeHz_21(theta)

## Models analysis

### P21

In [14]:
parlabels_21 = ["M","H_0",'q_0','j_0']
parlabtex_21 = ["$M$","$H_0$",'$q_0$','$j_0$']
parnames_21 = ['M','H0','q0','j0']
par0_21=[-19.2, 73.03398760350208, -0.5280171275112104, 1]
ndim_21 = len(par0_21)

#### CC+SH0ES+Pantheon+

In [15]:
par_mlHzPS_21 = par0_21

In [16]:
resultHzPS_21 = find_bestfit(getpar_PSHz_21, parnames_21, par_mlHzPS_21)
par_mlHzPS_21=resultHzPS_21['x']
print('\n',par_mlHzPS_21,'\n')
print(resultHzPS_21,'\n')

/home/nirk20/jupyter_venv/lib/python3.12/site-packages/scipy/optimize/_numdiff.py:596: RuntimeWarning: invalid value encountered in subtract
  df = fun(x1) - f0


Maximum likelihood result:
M  =  -19.253427502449398
H0  =  73.32840195934713
q0  =  -0.5798951804861703
j0  =  2.2694028184834334
chi2min = 1469.5326661208758
tempo total: 3.883 seg

 [-19.2534275   73.32840196  -0.57989518   2.26940282] 

       message: Optimization terminated successfully.
       success: True
        status: 0
           fun: 1469.5326661208758
             x: [-1.925e+01  7.333e+01 -5.799e-01  2.269e+00]
           nit: 217
          nfev: 361
 final_simplex: (array([[-1.925e+01,  7.333e+01, -5.799e-01,  2.269e+00],
                       [-1.925e+01,  7.333e+01, -5.799e-01,  2.269e+00],
                       ...,
                       [-1.925e+01,  7.333e+01, -5.799e-01,  2.269e+00],
                       [-1.925e+01,  7.333e+01, -5.799e-01,  2.269e+00]],
                      shape=(5, 4)), array([ 1.470e+03,  1.470e+03,  1.470e+03,  1.470e+03,
                        1.470e+03])) 



In [17]:
info_PSHz21 = {
    "likelihood": {
        "lnlikehz": lnprobPSHz_21
    },

    "params": {
        "M": {"prior": {"min": -19.5, "max": 19.5}, "proposal": par_mlHzPS_21[0], "latex": "M"},
        "h0": {"prior": {"min": 40.0, "max": 100.0}, "proposal": par_mlHzPS_21[1], "latex": r"H_{0}"},
        "q0": {"prior": {"min": -2, "max": 2}, "proposal": par_mlHzPS_21[2], "latex": r"q_{0}"},
        "j0": {"prior": {"min": -5, "max": 5}, "proposal": par_mlHzPS_21[3], "latex": r"j_{0}"}
    },

    "sampler": {
        "polychord": {
            "nlive": 100
        }
    },

    "output": "chains/P21/PSHz_21"
}

info_post_PSHz21 = {
    "output": "chains/P21/PSHz_21",
    "post": {
        "skip_samples": 0.3,
        "suffix": "_post"
    }
}

In [19]:
_, sampler_PSHz_21 = run_cobaya(info_PSHz21, info_post_PSHz21)

----- INICIANDO SAMPLER -----
[output] Output to be read-from/written-into folder 'chains/P21', with prefix 'PSHz_21'
[lnlikehz] Initialized external likelihood.
[polychord] `pypolychord` module loaded successfully from /home/nirk20/jupyter_venv/lib/python3.12/site-packages/pypolychord
[polychord] Storing raw PolyChord output in 'chains/P21/PSHz_21_polychord_raw'.
[model] Measuring speeds... (this may take a few seconds)
[model] Setting measured speeds (per sec): {lnlikehz: 128.0}
[polychord] Parameter blocks and their oversampling factors:
[polychord] * 1 : ['M', 'h0', 'q0', 'j0']
[polychord] Calling PolyChord...
PolyChord: MPI is already initilised, not initialising, and will not finalize

PolyChord: Next Generation Nested Sampling
copyright: Will Handley, Mike Hobson & Anthony Lasenby
  version: 1.22.1
  release: 10th Jan 2024
    email: wh260@mrao.cam.ac.uk

Run Settings
nlive    :     100
nDims    :       4
nDerived :       2
Doing Clustering
Synchronous parallelisation
Generating

#### CC+Pantheon+

In [15]:
par_mlHzPP_21 = par0_21

In [17]:
resultHzPP_21 = find_bestfit(getpar_PPHz_21, parnames_21, par_mlHzPP_21)
par_mlHzPP_21=resultHzPP_21['x']
print('\n',par_mlHzPP_21,'\n')
print(resultHzPP_21,'\n')

Maximum likelihood result:
M  =  -12.351193838336917
H0  =  69.1043329589236
q0  =  -0.5961098582696993
j0  =  2.4549315819076085
chi2min = 1418.4627051569305
tempo total: 3.030 seg

 [-12.35119384  69.10433296  -0.59610986   2.45493158] 

       message: Optimization terminated successfully.
       success: True
        status: 0
           fun: 1418.4627051569305
             x: [-1.235e+01  6.910e+01 -5.961e-01  2.455e+00]
           nit: 127
          nfev: 224
 final_simplex: (array([[-1.235e+01,  6.910e+01, -5.961e-01,  2.455e+00],
                       [-1.235e+01,  6.910e+01, -5.961e-01,  2.455e+00],
                       ...,
                       [-1.235e+01,  6.910e+01, -5.961e-01,  2.455e+00],
                       [-1.235e+01,  6.910e+01, -5.961e-01,  2.455e+00]],
                      shape=(5, 4)), array([ 1.418e+03,  1.418e+03,  1.418e+03,  1.418e+03,
                        1.418e+03])) 



In [18]:
info_PPHz21 = {
    "likelihood": {
        "lnlikehz": lnprobPPHz_21
    },

    "params": {
        "M": {"prior": {"min": -19.5, "max": 19.5}, "proposal": par_mlHzPP_21[0], "latex": "M"},
        "h0": {"prior": {"min": 40.0, "max": 100.0}, "proposal": par_mlHzPP_21[1], "latex": r"H_{0}"},
        "q0": {"prior": {"min": -2, "max": 2}, "proposal": par_mlHzPP_21[2], "latex": r"q_{0}"},
        "j0": {"prior": {"min": -5, "max": 5}, "proposal": par_mlHzPP_21[3], "latex": r"j_{0}"}
    },

    "sampler": {
        "polychord": {
            "nlive": 100
        }
    },

    "output": "chains/P21/PPHz_21"
}

info_post_PPHz21 = {
    "output": "chains/P21/PPHz_21",
    "post": {
        "skip_samples": 0.3,
        "suffix": "_post"
    }
}

In [20]:
_, sampler_PPHz_21 = run_cobaya(info_PPHz21, info_post_PPHz21)

----- INICIANDO SAMPLER -----
[output] Output to be read-from/written-into folder 'chains/P21', with prefix 'PPHz_21'
[lnlikehz] Initialized external likelihood.
[polychord] `pypolychord` module loaded successfully from /home/nirk20/jupyter_venv/lib/python3.12/site-packages/pypolychord
[polychord] Storing raw PolyChord output in 'chains/P21/PPHz_21_polychord_raw'.
[model] Measuring speeds... (this may take a few seconds)
[model] Setting measured speeds (per sec): {lnlikehz: 57.3}
[polychord] Parameter blocks and their oversampling factors:
[polychord] * 1 : ['M', 'h0', 'q0', 'j0']
[polychord] Calling PolyChord...
PolyChord: MPI is already initilised, not initialising, and will not finalize

PolyChord: Next Generation Nested Sampling
copyright: Will Handley, Mike Hobson & Anthony Lasenby
  version: 1.22.1
  release: 10th Jan 2024
    email: wh260@mrao.cam.ac.uk

Run Settings
nlive    :     100
nDims    :       4
nDerived :       2
Doing Clustering
Synchronous parallelisation
Generating 

### P22

In [14]:
parlabels_22 = ["M","H_0",'q_0','j_0', 's_0']
parlabtex_22 = ["$M$","$H_0$",'$q_0$','$j_0$', '$s_0$']
parnames_22 = ['M','H0','q0','j0', 's0']
par0_22=[-19.2, 73.03398760350208, -0.5280171275112104, 1, 4]
ndim_22 = len(par0_22)

#### CC+SH0ES+Pantheon+

In [15]:
par_mlHzPS_22 = par0_22

In [16]:
resultHzPS_22 = find_bestfit(getpar_PSHz_22, parnames_22, par_mlHzPS_22)
par_mlHzPS_22 = resultHzPS_22['x']
print('\n',par_mlHzPS_22,'\n')
print(resultHzPS_22,'\n')

/home/nirk20/jupyter_venv/lib/python3.12/site-packages/scipy/optimize/_numdiff.py:596: RuntimeWarning: invalid value encountered in subtract
  df = fun(x1) - f0
/home/nirk20/jupyter_venv/lib/python3.12/site-packages/scipy/optimize/_numdiff.py:596: RuntimeWarning: invalid value encountered in subtract
  df = fun(x1) - f0


Maximum likelihood result:
M  =  -19.23608617349228
H0  =  6.492175289974673
q0  =  -0.4960879698744289
j0  =  1.606348009931004
s0  =  3.785090874213947
chi2min = 1844.942526901966
tempo total: 8.923 seg

 [-19.23608617   6.49217529  -0.49608797   1.60634801   3.78509087] 

       message: Optimization terminated successfully.
       success: True
        status: 0
           fun: 1844.942526901966
             x: [-1.924e+01  6.492e+00 -4.961e-01  1.606e+00  3.785e+00]
           nit: 826
          nfev: 1323
 final_simplex: (array([[-1.924e+01,  6.492e+00, ...,  1.606e+00,
                         3.785e+00],
                       [-1.924e+01,  6.492e+00, ...,  1.606e+00,
                         3.785e+00],
                       ...,
                       [-1.924e+01,  6.492e+00, ...,  1.606e+00,
                         3.785e+00],
                       [-1.924e+01,  6.492e+00, ...,  1.606e+00,
                         3.785e+00]], shape=(6, 5)), array([ 1.845e+03,  1.845e+03,

In [19]:
info_PSHz22 = {
    "likelihood": {
        "lnlikehz": lnprobPSHz_22
    },

    "params": {
        "M": {"prior": {"min": -19.5, "max": 19.5}, "proposal": par_mlHzPS_22[0], "latex": "M"},
        "h0": {"prior": {"min": 40.0, "max": 100.0}, "proposal": par_mlHzPS_22[1], "latex": r"H_{0}"},
        "q0": {"prior": {"min": -2, "max": 2}, "proposal": par_mlHzPS_22[2], "latex": r"q_{0}"},
        "j0": {"prior": {"min": -5, "max": 5}, "proposal": par_mlHzPS_22[3], "latex": r"j_{0}"},
        "s0": {"prior": {"min": -10, "max": 10}, "proposal": par_mlHzPS_22[4], "latex": r"s_{0}"}
    },

    "sampler": {
        "polychord": {
            "nlive":500
        }
    },

    "output": "chains/P22/PS/PSHz_22"
}

info_post_PSHz22 = {
    "output": "chains/P22/PS/PSHz_22",
    "post": {
        "skip_samples": 0.3,
        "suffix": "_post"
    }
}

In [20]:
_, sampler_PSHz_22 = run_cobaya(info_PSHz22, info_post_PSHz22)

----- INICIANDO SAMPLER -----
[output] Output to be read-from/written-into folder 'chains/P22/PS', with prefix 'PSHz_22'
[lnlikehz] Initialized external likelihood.
[polychord] `pypolychord` module loaded successfully from /home/nirk20/jupyter_venv/lib/python3.12/site-packages/pypolychord
[polychord] Storing raw PolyChord output in 'chains/P22/PS/PSHz_22_polychord_raw'.
[model] Measuring speeds... (this may take a few seconds)
[model] Setting measured speeds (per sec): {lnlikehz: 60.2}
[polychord] Parameter blocks and their oversampling factors:
[polychord] * 1 : ['M', 'h0', 'q0', 'j0', 's0']
[polychord] Calling PolyChord...
PolyChord: MPI is already initilised, not initialising, and will not finalize

PolyChord: Next Generation Nested Sampling
copyright: Will Handley, Mike Hobson & Anthony Lasenby
  version: 1.22.1
  release: 10th Jan 2024
    email: wh260@mrao.cam.ac.uk

Run Settings
nlive    :     500
nDims    :       5
nDerived :       2
Doing Clustering
Synchronous parallelisation

#### CC+Pantheon+

In [15]:
par_mlHzPP_22 = par0_22

In [16]:
resultHzPP_22 = find_bestfit(getpar_PPHz_22, parnames_22, par_mlHzPP_22)
par_mlHzPP_22 = resultHzPP_22['x']
print('\n',par_mlHzPP_22,'\n')
print(resultHzPP_22,'\n')

Maximum likelihood result:
M  =  -63.140614053537604
H0  =  73.27558007430063
q0  =  -0.6878727916300897
j0  =  3.483492079808695
s0  =  9.088470921766225
chi2min = 1432.0007735232118
tempo total: 3.758 seg

 [-63.14061405  73.27558007  -0.68787279   3.48349208   9.08847092] 

       message: Optimization terminated successfully.
       success: True
        status: 0
           fun: 1432.0007735232118
             x: [-6.314e+01  7.328e+01 -6.879e-01  3.483e+00  9.088e+00]
           nit: 213
          nfev: 379
 final_simplex: (array([[-6.314e+01,  7.328e+01, ...,  3.483e+00,
                         9.088e+00],
                       [-6.314e+01,  7.328e+01, ...,  3.483e+00,
                         9.088e+00],
                       ...,
                       [-6.314e+01,  7.328e+01, ...,  3.483e+00,
                         9.088e+00],
                       [-6.314e+01,  7.328e+01, ...,  3.483e+00,
                         9.088e+00]], shape=(6, 5)), array([ 1.432e+03,  1.432e+0

In [17]:
info_PPHz22 = {
    "likelihood": {
        "lnlikehz": lnprobPPHz_22
    },

    "params": {
        "M": {"prior": {"min": -19.5, "max": 19.5}, "proposal": par_mlHzPP_22[0], "latex": "M"},
        "h0": {"prior": {"min": 40.0, "max": 100.0}, "proposal": par_mlHzPP_22[1], "latex": r"H_{0}"},
        "q0": {"prior": {"min": -2, "max": 2}, "proposal": par_mlHzPP_22[2], "latex": r"q_{0}"},
        "j0": {"prior": {"min": -5, "max": 5}, "proposal": par_mlHzPP_22[3], "latex": r"j_{0}"},
        "s0": {"prior": {"min": -10, "max": 10}, "proposal": par_mlHzPP_22[4], "latex": r"s_{0}"}
    },

    "sampler": {
        "polychord": {
            "nlive": 100
        }
    },

    "output": "chains/P22/PPHz_22"
}

info_post_PPHz22 = {
    "output": "chains/P22/PPHz_22",
    "post": {
        "skip_samples": 0.3,
        "suffix": "_post"
    }
}

In [18]:
_, sampler_PPHz_22 = run_cobaya(info_PPHz22, info_post_PPHz22)

----- INICIANDO SAMPLER -----
[output] Output to be read-from/written-into folder 'chains/P22', with prefix 'PPHz_22'
[lnlikehz] Initialized external likelihood.
[polychord] `pypolychord` module loaded successfully from /home/nirk20/jupyter_venv/lib/python3.12/site-packages/pypolychord
[polychord] Storing raw PolyChord output in 'chains/P22/PPHz_22_polychord_raw'.
[model] Measuring speeds... (this may take a few seconds)
[model] Setting measured speeds (per sec): {lnlikehz: 135.0}
[polychord] Parameter blocks and their oversampling factors:
[polychord] * 1 : ['M', 'h0', 'q0', 'j0', 's0']
[polychord] Calling PolyChord...
PolyChord: MPI is already initilised, not initialising, and will not finalize

PolyChord: Next Generation Nested Sampling
copyright: Will Handley, Mike Hobson & Anthony Lasenby
  version: 1.22.1
  release: 10th Jan 2024
    email: wh260@mrao.cam.ac.uk

Run Settings
nlive    :     100
nDims    :       5
nDerived :       2
Doing Clustering
Synchronous parallelisation
Gene

### P31

#### CC+SH0ES+Pantheon+

In [15]:
par_mlHzPS_31 = par0_22

In [16]:
resultHzPS_31 = find_bestfit(getpar_PSHz_31, parnames_22, par_mlHzPS_31)
par_mlHzPS_31 = resultHzPS_31['x']
print('\n',par_mlHzPS_31,'\n')
print(resultHzPS_31,'\n')

/home/nirk20/jupyter_venv/lib/python3.12/site-packages/scipy/optimize/_numdiff.py:596: RuntimeWarning: invalid value encountered in subtract
  df = fun(x1) - f0
/tmp/ipykernel_86239/3288264863.py:34: IntegrationWarning: The maximum number of subdivisions (50) has been achieved.
  If increasing the limit yields no improvement it is advised to analyze 
  the integrand in order to determine the difficulties.  If the position of a 
  local difficulty can be determined (singularity, discontinuity) one will 
  probably gain from splitting up the interval and calling the integrator 
  on the subranges.  Perhaps a special-purpose integrator should be used.
  dc[i], _ = quad(EzP31, 0, z[i], args=(theta))
/tmp/ipykernel_86239/3288264863.py:34: IntegrationWarning: The algorithm does not converge.  Roundoff error is detected
  in the extrapolation table.  It is assumed that the requested tolerance
  cannot be achieved, and that the returned result (if full_output = 1) is 
  the best which can be o

Maximum likelihood result:
M  =  -28.404419746059546
H0  =  41.50507621952556
q0  =  -1.5156357687444546
j0  =  1.5202005587199514
s0  =  -2.7377241804170707
chi2min = 100004.2236439458
tempo total: 1390.853 seg

 [-28.40441975  41.50507622  -1.51563577   1.52020056  -2.73772418] 

       message: Optimization terminated successfully.
       success: True
        status: 0
           fun: 100004.2236439458
             x: [-2.840e+01  4.151e+01 -1.516e+00  1.520e+00 -2.738e+00]
           nit: 886
          nfev: 1540
 final_simplex: (array([[-2.840e+01,  4.151e+01, ...,  1.520e+00,
                        -2.738e+00],
                       [-2.840e+01,  4.151e+01, ...,  1.520e+00,
                        -2.738e+00],
                       ...,
                       [-2.840e+01,  4.151e+01, ...,  1.520e+00,
                        -2.738e+00],
                       [-2.840e+01,  4.151e+01, ...,  1.520e+00,
                        -2.738e+00]], shape=(6, 5)), array([ 1.000e+05,  1.0

In [17]:
info_PSHz31 = {
    "likelihood": {
        "lnlikehz": lnprobPSHz_31
    },

    "params": {
        "M": {"prior": {"min": -19.5, "max": 19.5}, "proposal": par_mlHzPS_31[0], "latex": "M"},
        "h0": {"prior": {"min": 40.0, "max": 100.0}, "proposal": par_mlHzPS_31[1], "latex": r"H_{0}"},
        "q0": {"prior": {"min": -2, "max": 2}, "proposal": par_mlHzPS_31[2], "latex": r"q_{0}"},
        "j0": {"prior": {"min": -5, "max": 5}, "proposal": par_mlHzPS_31[3], "latex": r"j_{0}"},
        "s0": {"prior": {"min": -10, "max": 10}, "proposal": par_mlHzPS_31[4], "latex": r"s_{0}"}
    },

    "sampler": {
        "polychord": {
            "nlive": 100
        }
    },

    "output": "chains/P31/PSHz_31"
}

info_post_PSHz31 = {
    "output": "chains/P31/PSHz_31",
    "post": {
        "skip_samples": 0.3,
        "suffix": "_post"
    }
}

In [19]:
_, sampler_PSHz_31 = run_cobaya(info_PSHz31, info_post_PSHz31)

----- INICIANDO SAMPLER -----
[output] Output to be read-from/written-into folder 'chains/P31', with prefix 'PSHz_31'
[lnlikehz] Initialized external likelihood.
[polychord] `pypolychord` module loaded successfully from /home/nirk20/jupyter_venv/lib/python3.12/site-packages/pypolychord
[polychord] Storing raw PolyChord output in 'chains/P31/PSHz_31_polychord_raw'.
[model] Measuring speeds... (this may take a few seconds)
[model] Setting measured speeds (per sec): {lnlikehz: 0.915}
[polychord] Parameter blocks and their oversampling factors:
[polychord] * 1 : ['M', 'h0', 'q0', 'j0', 's0']
[polychord] Calling PolyChord...
PolyChord: MPI is already initilised, not initialising, and will not finalize

PolyChord: Next Generation Nested Sampling
copyright: Will Handley, Mike Hobson & Anthony Lasenby
  version: 1.22.1
  release: 10th Jan 2024
    email: wh260@mrao.cam.ac.uk

Run Settings
nlive    :     100
nDims    :       5
nDerived :       2
Doing Clustering
Synchronous parallelisation
Gene

/tmp/ipykernel_86239/3288264863.py:34: IntegrationWarning: The maximum number of subdivisions (50) has been achieved.
  If increasing the limit yields no improvement it is advised to analyze 
  the integrand in order to determine the difficulties.  If the position of a 
  local difficulty can be determined (singularity, discontinuity) one will 
  probably gain from splitting up the interval and calling the integrator 
  on the subranges.  Perhaps a special-purpose integrator should be used.
  dc[i], _ = quad(EzP31, 0, z[i], args=(theta))
/tmp/ipykernel_86239/3288264863.py:34: IntegrationWarning: The algorithm does not converge.  Roundoff error is detected
  in the extrapolation table.  It is assumed that the requested tolerance
  cannot be achieved, and that the returned result (if full_output = 1) is 
  the best which can be obtained.
  dc[i], _ = quad(EzP31, 0, z[i], args=(theta))
/tmp/ipykernel_86239/3288264863.py:34: IntegrationWarning: The algorithm does not converge.  Roundoff er


all live points generated

number of repeats:           10
started sampling



/tmp/ipykernel_86239/3288264863.py:34: IntegrationWarning: The maximum number of subdivisions (50) has been achieved.
  If increasing the limit yields no improvement it is advised to analyze 
  the integrand in order to determine the difficulties.  If the position of a 
  local difficulty can be determined (singularity, discontinuity) one will 
  probably gain from splitting up the interval and calling the integrator 
  on the subranges.  Perhaps a special-purpose integrator should be used.
  dc[i], _ = quad(EzP31, 0, z[i], args=(theta))
/tmp/ipykernel_86239/3288264863.py:34: IntegrationWarning: The algorithm does not converge.  Roundoff error is detected
  in the extrapolation table.  It is assumed that the requested tolerance
  cannot be achieved, and that the returned result (if full_output = 1) is 
  the best which can be obtained.
  dc[i], _ = quad(EzP31, 0, z[i], args=(theta))
/tmp/ipykernel_86239/3288264863.py:12: RuntimeWarning: divide by zero encountered in scalar divide
  H =

Polychord Warning: Non deterministic loglikelihood


...

Polychord Warning: Non deterministic loglikelihood


/tmp/ipykernel_86239/3288264863.py:34: IntegrationWarning: The maximum number of subdivisions (50) has been achieved.
  If increasing the limit yields no improvement it is advised to analyze 
  the integrand in order to determine the difficulties.  If the position of a 
  local difficulty can be determined (singularity, discontinuity) one will 
  probably gain from splitting up the interval and calling the integrator 
  on the subranges.  Perhaps a special-purpose integrator should be used.
  dc[i], _ = quad(EzP31, 0, z[i], args=(theta))
/tmp/ipykernel_86239/3288264863.py:34: IntegrationWarning: The algorithm does not converge.  Roundoff error is detected
  in the extrapolation table.  It is assumed that the requested tolerance
  cannot be achieved, and that the returned result (if full_output = 1) is 
  the best which can be obtained.
  dc[i], _ = quad(EzP31, 0, z[i], args=(theta))
/tmp/ipykernel_86239/3288264863.py:34: IntegrationWarning: The integral is probably divergent, or slowly

Polychord Warning: Non deterministic loglikelihood


/tmp/ipykernel_86239/3288264863.py:34: IntegrationWarning: The maximum number of subdivisions (50) has been achieved.
  If increasing the limit yields no improvement it is advised to analyze 
  the integrand in order to determine the difficulties.  If the position of a 
  local difficulty can be determined (singularity, discontinuity) one will 
  probably gain from splitting up the interval and calling the integrator 
  on the subranges.  Perhaps a special-purpose integrator should be used.
  dc[i], _ = quad(EzP31, 0, z[i], args=(theta))
/tmp/ipykernel_86239/3288264863.py:34: IntegrationWarning: The algorithm does not converge.  Roundoff error is detected
  in the extrapolation table.  It is assumed that the requested tolerance
  cannot be achieved, and that the returned result (if full_output = 1) is 
  the best which can be obtained.
  dc[i], _ = quad(EzP31, 0, z[i], args=(theta))
/tmp/ipykernel_86239/3288264863.py:12: RuntimeWarning: divide by zero encountered in scalar divide
  H =

KeyboardInterrupt: 

#### CC+Pantheon+

In [ ]:
par_mlHzPP_31 = par0_22

In [ ]:
resultHzPP_31 = find_bestfit(getpar_PPHz_31, parnames_22, par_mlHzPP_31)
par_mlHzPP_31 = resultHzPP_31['x']
print('\n',par_mlHzPP_31,'\n')
print(resultHzPP_31,'\n')

In [ ]:
info_PPHz31 = {
    "likelihood": {
        "lnlikehz": lnprobPPHz_31
    },

    "params": {
        "M": {"prior": {"min": -19.5, "max": 19.5}, "proposal": par_mlHzPP_31[0], "latex": "M"},
        "h0": {"prior": {"min": 40.0, "max": 100.0}, "proposal": par_mlHzPP_31[1], "latex": r"H_{0}"},
        "q0": {"prior": {"min": -2, "max": 2}, "proposal": par_mlHzPP_31[2], "latex": r"q_{0}"},
        "j0": {"prior": {"min": -5, "max": 5}, "proposal": par_mlHzPP_31[3], "latex": r"j_{0}"},
        "s0": {"prior": {"min": -10, "max": 10}, "proposal": par_mlHzPP_31[4], "latex": r"s_{0}"}
    },

    "sampler": {
        "polychord": {
            "nlive": 100
        }
    },

    "output": "chains/P31/PPHz_31"
}

info_post_PPHz31 = {
    "output": "chains/P31/PPHz_31",
    "post": {
        "skip_samples": 0.3,
        "suffix": "_post"
    }
}

In [ ]:
_, sampler_PPHz_31 = run_cobaya(info_PPHz31, info_post_PPHz31)